### Instituto Tecnologico de Costa Rica (ITCR)
### Sede Interuniversitaria de Alajuela
### Escuela de Computacion
### Curso: Inteligencia Artificial
### Estudiantes: 

 - Brandon Ledezma Fernández - 2018185574
 - Walter Morales Vásquez - 2018212846

### Profesora:
 
 - Maria Auxiliadora Mora


# Tarea Programada Número 4
---
#### Introducción:
En este trabajo práctico se aplicarán conceptos básicos de aprendizaje automático
utilizando redes neuronales recurrentes para resolver problemas que involucran el
procesamiento de lenguaje natural.

El o los estudiantes deberán realizar dos ejercicios. El primero consiste en implementar
una red neuronal recurrente aplicada a un problema de clasificación de textos de opinión
sobre prendas de vestir de mujer. El segundo ejercicio consiste en reconocer nombres de
entidades en textos (NER, Named-Entity Recognition)

El objetivo del trabajo es poner en práctica las habilidades de investigación y el
conocimiento adquirido durante el curso sobre redes neuronales por medio de
ejercicios prácticos que permitan al estudiante experimentar con el aprendizaje profundo. 

In [56]:
# Tratamiento de datos
# ==============================================================================
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.model_selection import train_test_split

## A. Clasificación   de   textos   con   redes   neuronales   recurrentes  LSTM utilizando Pytorch.

Se desea que, dado un comentario de revisión de una prenda de vestir, predecir la calificación dada por el comprador. La calificación toma valores enteros entre 1 y 5, donde 1 corresponde a la peor calificación y 5 a la mejor.

Datos: Utilice los datos de evaluación de prendas de vestir de mujer disponibles en Kaggle (nicapotato, 2018) para:

1. Cargue y prepare los datos para ser introducidos a la red LSTM.

In [77]:
nRowsRead = 1000

# Se carga el archivo con los datos solicitados (defaultofcredit.csv) y se define
# a la columna "default_payment_next_month" como la objetivo.
reviews = pd.read_csv('./data/Womens Clothing E-Commerce Reviews.csv', nrows=nRowsRead)
#reviews = pd.read_csv('./data/a.csv')

# Delete missing observations for following variables
for x in ["Clothing ID","Age","Title","Review Text","Rating","Recommended IND","Positive Feedback Count","Division Name","Department Name","Class Name"]:
    reviews = reviews[reviews[x].notnull()]
    
X = reviews.drop(columns = reviews.columns[3:5])
X = pd.get_dummies(X)
y = reviews['Review Text']
y = [elem.split() for elem in y]

word_to_ix = {}
for review in y:
    for word in review:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
            
X

,Unnamed: 0,Clothing ID,Age,Rating,Recommended IND,Positive Feedback Count,Division Name_General,Division Name_General Petite,Division Name_Initmates,Department Name_Bottoms,...,Class Name_Legwear,Class Name_Lounge,Class Name_Outerwear,Class Name_Pants,Class Name_Shorts,Class Name_Skirts,Class Name_Sleep,Class Name_Sweaters,Class Name_Swim,Class Name_Trend
2,2,1077,60,3,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,1049,50,5,1,0,0,1,0,1,...,0,0,0,1,0,0,0,0,0,0
4,4,847,47,5,1,6,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,5,1080,49,2,0,4,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,6,858,39,5,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
993,993,1094,48,3,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996,996,936,37,5,1,8,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
997,997,936,36,5,1,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
998,998,854,29,5,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


2. Utilizando PyTorch defina una red recurrente LSTM para procesar el conjunto de datos y clasificar los comentarios de usuario. 

In [ ]:
# Definición del modelo

# El modelo es una clase que debe heredar de nn.Module
class LSTMTagger(nn.Module):
    
    # Incialización del modelo
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
 

        # Primero se pasa la entrada a través de una capa Embedding. 
        # Esta capa construye una representación de los tokens de 
        # un texto donde las palabras que tienen el mismo significado 
        # tienen una representación similar.
        
        # Esta capa captura mejor el contexto y son espacialmente 
        # más eficientes que las representaciones vectoriales (one-hot vector).
        # En Pytorch, se usa el módulo nn.Embedding para crear esta capa, 
        # que toma el tamaño del vocabulario y la longitud deseada del vector 
        # de palabras como entrada. 
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # El LSTM toma word_embeddings como entrada y genera estados ocultos
        # con dimensionalidad hidden_dim.  
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # La capa lineal mapea el espacio de estado oculto 
        # al espacio de etiquetas
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        # Pase hacia adelante de la red. 
        # Parámetros:
        #    sentence: la oración a procesar
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))

        # Se utiliza softmax para devolver un peso por etiqueta
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

# Instanciación del modelo, definición de la función de pérdida y del optimizador   

# Hiperparámetros de la red
# Valores generalmente altos (32 o 64 dimensiones).
# Se definen pequeños, para ver cómo cambian los pesos durante el entrenamiento.

EMBEDDING_DIM = 6
HIDDEN_DIM = 6

# Instancia del modelo
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))

# Función de pérdida: Negative Log Likelihood Loss (NLLL). 
# Útil para problemas de clasificacion con C clases.
loss_function = nn.NLLLoss()

# Optimizador Stochastic Gradient Descent  
optimizer = optim.SGD(model.parameters(), lr=0.1)


3. Separe las muestras en datos de entrenamiento y evaluación y entrene el modelo. 

In [61]:
X_train, X_test, y_train, y_test = train_test_split(
                                        X,
                                        y.values.reshape(-1,1),
                                        train_size   = 0.7,
                                        random_state = 1234,
                                        shuffle      = True
                                    )

# print('X_train', X_train.values)
# print('X_test', X_test.values)
# print('y_train', y_train)
# print('y_test', y_test)

#print('X.values.tolist()', X.values.tolist())

test = torch.tensor(y.values.tolist())

# X_train = torch.tensor(X_train.values.astype(np.float32)) 
# X_test = torch.tensor(X_test.values.astype(np.float32)) 

# y_train = torch.tensor(y_train.values.astype(np.float32))
# y_test = torch.tensor(y_test.values.astype(np.float32))

# train_target = torch.tensor(train['Target'].values.astype(np.float32))
# train = torch.tensor(train.drop('Target', axis = 1).values.astype(np.float32)) 
# train_tensor = data_utils.TensorDataset(train, train_target) 
# train_loader = data_utils.DataLoader(dataset = train_tensor, batch_size = batch_size, shuffle = True)

ValueError: too many dimensions 'str'

In [ ]:
# Preparación de los datos 
def prepare_sequence(seq, to_ix):
    # Prepara tensores de indices de palabras a partir de una oración.
    # Parámetros:
    #   seq: oración
    #   to_ix: diccionario de palabras.
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

# Entrenar el modelo 

# Valores antes de entrenar
# El elemento i, j de la salida es la puntuación entre la etiqueta j para la palabra i.
with torch.no_grad():
    inputs = prepare_sequence(X_train[0][0], word_to_ix)
    tag_scores = model(inputs)
    
    print(training_data[0][0])
    
    # Clasificación    
    print(tag_scores)

# Corridas o épocas
for epoch in range(500):  
    for sentence, tags in training_data:
        ## Paso 1. Pytorch acumula los gradientes.
        # Es necesario limpiarlos
        model.zero_grad()

        # Paso 2. Se preparan las entradas, es decir, se convierten a
        # tensores de índices de palabras.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Paso 3. Se genera la predicción (forward pass).
        tag_scores = model(sentence_in)

        # Paso 4. se calcula la pérdida, los gradientes, y se actualizan los 
        # parámetros por medio del optimizador.
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# Despligue de la puntuación luego del entrenamiento
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
   
    print("Resultados luego del entrenamiento para la primera frase")
    # Las palabras en una oración se pueden etiquetar de tres formas.
    # La primera oración tiene 4 palabras "El perro come manzana"
    # por eso el tensor de salida tiene 4 elementos. 
    # Cada elemento es un vector de pesos que indica cuál etiqueta tiene más
    # posibilidad de estar asociada a la palabra. Es decir hay que calcular 
    # la posición del valor máximo
    print(tag_scores)

4. Evalúe el modelo resultante utilizando una matriz de confusión y métricas extraídasa partir de esta (ie. precisión, exhaustividad y F1). 

5. Genere y documente sus conclusiones (incluya al menos cuatro conclusionesimportantes). 

Referencias
----------------
*

## B. Reconocimiento   de   nombres   de   entidades   (NER,   Name   EntityRecognition) con redes neuronales recurrentes utilizando Pytorch.

El reconocimiento de nombres de entidades (NER)  es  el proceso  de identificar ycategorizar elementos clave (ej. entidades) en el texto. Una entidad puede ser cualquierpalabra o secuencia de palabras que se refieren a unapersona, animal, sitio o cosa (ej.empresa, región geográfica, objeto). Cada entidad detectada se clasifica en una categoríapredeterminada.  Normalmente, NER se aborda como un problema de etiquetado desecuencias. Una explicación muy detallada de porqué es importante extraer entidades delos textos se encuentra en (Monge, 2020).

Los algoritmos de extracción de entidades pueden únicamente detectar la presencia deuna entidad y marcarla como tal o pueden detectar y clasificar cada entidad queencuentran.

Ejemplo: En una oración como “arbusto de 2 m.  flores lila.”.  Cada palabra representa untoken donde “arbusto” y “flores” son los elementos de interés amarcar. El etiquetado“token inicial- token interno” es una forma común de indicar dónde comienzan y terminanlas entidades. En el ejemplo anterior la etiqueta sería “B O O O B O” donde B representael inicio de la entidad y O cualquier otro token. Para la oración “botones florales rosados.”la etiqueta estaría dada por “B I O” donde “B” marca el token inicio de la entidad e “I” losotros tokens que son parte de esta, es decir “B I” delimita la entidad “botones florales”. 

Otra forma de marcar y etiquetar, es además de delimitar la entidad, asignar a estalaclase a la que corresponde, por ejemplo: empresa, ciudad, persona, entre otros. Para elpresente ejercicio se va a utilizar este enfoque.

Utilice los datos para reconocer y clasificar nombres de entidadescompartidos en Kagglepor (Ranjan, 2020) para:

1. Cargue y prepare los datos para ser introducidos a la red recurrente.

2. Utilizando PyTorch defina una red recurrente LSTM para procesar localizary clasificar las entidades presentes en el texto (como la vista en clase). 

3. Separe las muestras en datos de entrenamiento y evaluación y entrene elmodelo. 

4. Evalúe el modelo resultante. Utilice la métrica propuesta por el InternationalWorkshop on Semantic Evaluation (SemEval), una explicación básica estádisponible en (Batista, 2018).  

5. Genere   y   documente   sus   conclusiones   (incluya   al   menos   cuatroconclusiones importantes).

Referencias
----------------
*